In [57]:
import math
import cv2
import numpy as np
import os
import shutil

In [71]:
#train, valid test 폴더 생성
for _ in ['train', 'valid', 'test', 'train_ori', 'valid_ori', 'test_ori']:
    if _ not in os.listdir(): os.mkdir(_)
    
Folder = [i for i in os.listdir() if os.path.isdir(i)]
classes = [i for i in Folder if i not in ['.ipynb_checkpoints', 'test', 'train', 'valid', 'train_ori', 'valid_ori', 'test_ori']]
classes

[]

In [60]:
#간단한 이름 변경
for i, store in enumerate(classes):
    for j, file in enumerate(os.listdir(store)):
        os.rename('./'+store+'/'+file, './'+store+'/{}_'.format(i+1)+store+'_{0:04d}.jpg'.format(j+1))

In [64]:
#300개 초과하는 파일 제거
for store in classes:
    pic_list = [i for i in os.listdir(store)]
    for i in pic_list[300:]:
        os.remove('./'+store+'/'+i)

In [67]:
print(f'{len(os.listdir(store))}개의 사진에서 60% train, 20% valid, 20% test로 나눔')

# 파일 이동시키기 및 폴더삭제
for store in classes:
    pic_list = [i for i in os.listdir(store)]
    for i in range(180):
        shutil.move('./'+store+'/'+pic_list[i],'train_ori')
    for i in range(180,240):
        shutil.move('./'+store+'/'+pic_list[i],'valid_ori')
    for i in range(240,300):
        shutil.move('./'+store+'/'+pic_list[i],'test_ori')
    os.rmdir(store)

300개의 사진에서 60% train, 20% valid, 20% test로 나눔


In [90]:
def imread(filename, flags=cv2.IMREAD_COLOR, dtype=np.uint8):
    try:
        n = np.fromfile(filename, dtype)
        img = cv2.imdecode(n, flags)
        return img
    except Exception as e:
        print(e)
        return None
    
def imwrite(filename, img, params=None):
    try:
        ext = os.path.splitext(filename)[1]
        result, n = cv2.imencode(ext, img, params)

        if result:
            with open(filename, mode='w+b') as f:
                n.tofile(f)
            return True
        else:
            return False
    except Exception as e:
        print(e)
        return False
    
def rotate_image(image, angle):

    # Get the image size
    # No that's not an error - NumPy stores image matricies backwards
    image_size = (image.shape[1], image.shape[0])
    image_center = tuple(np.array(image_size) / 2)

    # Convert the OpenCV 3x2 rotation matrix to 3x3
    rot_mat = np.vstack(
        [cv2.getRotationMatrix2D(image_center, angle, 1.0), [0, 0, 1]]
    )

    rot_mat_notranslate = np.matrix(rot_mat[0:2, 0:2])

    # Shorthand for below calcs
    image_w2 = image_size[0] * 0.5
    image_h2 = image_size[1] * 0.5

    # Obtain the rotated coordinates of the image corners
    rotated_coords = [
        (np.array([-image_w2,  image_h2]) * rot_mat_notranslate).A[0],
        (np.array([ image_w2,  image_h2]) * rot_mat_notranslate).A[0],
        (np.array([-image_w2, -image_h2]) * rot_mat_notranslate).A[0],
        (np.array([ image_w2, -image_h2]) * rot_mat_notranslate).A[0]
    ]

    # Find the size of the new image
    x_coords = [pt[0] for pt in rotated_coords]
    x_pos = [x for x in x_coords if x > 0]
    x_neg = [x for x in x_coords if x < 0]

    y_coords = [pt[1] for pt in rotated_coords]
    y_pos = [y for y in y_coords if y > 0]
    y_neg = [y for y in y_coords if y < 0]

    right_bound = max(x_pos)
    left_bound = min(x_neg)
    top_bound = max(y_pos)
    bot_bound = min(y_neg)

    new_w = int(abs(right_bound - left_bound))
    new_h = int(abs(top_bound - bot_bound))

    # We require a translation matrix to keep the image centred
    trans_mat = np.matrix([
        [1, 0, int(new_w * 0.5 - image_w2)],
        [0, 1, int(new_h * 0.5 - image_h2)],
        [0, 0, 1]
    ])

    # Compute the tranform for the combined rotation and translation
    affine_mat = (np.matrix(trans_mat) * np.matrix(rot_mat))[0:2, :]

    # Apply the transform
    result = cv2.warpAffine(
        image,
        affine_mat,
        (new_w, new_h),
        flags=cv2.INTER_LINEAR
    )

    return result

def largest_rotated_rect(w, h, angle):

    quadrant = int(math.floor(angle / (math.pi / 2))) & 3
    sign_alpha = angle if ((quadrant & 1) == 0) else math.pi - angle
    alpha = (sign_alpha % math.pi + math.pi) % math.pi

    bb_w = w * math.cos(alpha) + h * math.sin(alpha)
    bb_h = w * math.sin(alpha) + h * math.cos(alpha)

    gamma = math.atan2(bb_w, bb_w) if (w < h) else math.atan2(bb_w, bb_w)

    delta = math.pi - alpha - gamma

    length = h if (w < h) else w

    d = length * math.cos(alpha)
    a = d * math.sin(alpha) / math.sin(delta)

    y = a * math.cos(gamma)
    x = y * math.tan(gamma)

    return (
        bb_w - 2 * x,
        bb_h - 2 * y
    )

def crop_around_center(image, width, height):

    image_size = (image.shape[1], image.shape[0])
    image_center = (int(image_size[0] * 0.5), int(image_size[1] * 0.5))

    if(width > image_size[0]):
        width = image_size[0]

    if(height > image_size[1]):
        height = image_size[1]

    x1 = int(image_center[0] - width * 0.5)
    x2 = int(image_center[0] + width * 0.5)
    y1 = int(image_center[1] - height * 0.5)
    y2 = int(image_center[1] + height * 0.5)

    return image[y1:y2, x1:x2]

def rotate_crop(image, angle, save_path, filename, resize=200):
    image_height, image_width = image.shape[0:2]
    image_rotated = rotate_image(image, angle)
    image_rotated_cropped = crop_around_center(image_rotated, *largest_rotated_rect(image_width, image_height, math.radians(angle)))
    
    #center crop
    height, width = image_rotated_cropped.shape[0:2]
    cubic = min(height, width)
    x = image_rotated_cropped.shape[1]/2 - cubic/2
    y = image_rotated_cropped.shape[0]/2 - cubic/2
    crop_img = image_rotated_cropped[int(y):int(y+cubic), int(x):int(x+cubic)]
    
    #크기변경을 위해 resize
    resize_img = cv2.resize(crop_img, (resize, resize))
    
    imwrite('./'+save_path+'/'+filename[:-4]+'{}_deg.jpg'.format(angle), resize_img)

In [86]:
angle = [i for i in range(0,180,10)]
for folder in ['train', 'valid', 'test']:
    for ang in angle:
        for filename in os.listdir(folder+'_ori/'):
            img = imread('./'+folder+'_ori/'+ filename)
            rotate_crop(img, ang, folder, filename)
        print(f'{ang} deg 작업 완료되었습니다.')

0 deg 작업 완료되었습니다.
10 deg 작업 완료되었습니다.
20 deg 작업 완료되었습니다.
30 deg 작업 완료되었습니다.
40 deg 작업 완료되었습니다.
50 deg 작업 완료되었습니다.
60 deg 작업 완료되었습니다.
70 deg 작업 완료되었습니다.
80 deg 작업 완료되었습니다.
90 deg 작업 완료되었습니다.
100 deg 작업 완료되었습니다.
110 deg 작업 완료되었습니다.
120 deg 작업 완료되었습니다.
130 deg 작업 완료되었습니다.


KeyboardInterrupt: 

In [ ]:
for i in ['train', 'valid', 'test']: print(f'{i} 이미지 개수 : {len(os.listdir(i))}')

In [89]:
filename[:-4]

'3_오스테리아57_0007'